# Form.io Apply Datamodel AI Workflow

This notebook documents the workflow to apply a hierarchical datamodel (XML) onto an existing Form.io JSON schema using an LLM. It is intended as instructional documentation rather than fully runnable code.


## Inputs and preparation

- Existing Form.io JSON (`currentJson`)
- Datamodel XML (from text or storage)
- Optional provider selection (`openai`, `anthropic`, `mistral`)


In [ ]:
# Illustrative placeholders — replace with your integration
CURRENT_JSON = {
  "display": "form",
  "components": [
    {"type": "textfield", "key": "firstname", "label": "First name"},
    {"type": "textfield", "key": "lastname", "label": "Last name"},
    {"type": "button", "action": "submit", "label": "Submit"}
  ]
}

DATAMODEL_XML = """
<DataModel>
  <Node name="Address" multiple="false">
    <Node name="Line" multiple="true" data-type="TEXT" />
  </Node>
</DataModel>
""".strip()



## System prompt (exact)

The following system prompt matches the one used by the server route.


In [ ]:
SYSTEM = """
You are a senior Form.io architect. You must merge a Form.io form JSON with a hierarchical XML datamodel.

Output contract:
- Return only valid Form.io JSON. No prose, no markdown fences.
- Preserve top-level {{ display: "form", components: [...] }}.
- For each datamodel node:
  - If multiple="true" or the node represents a list, use a Form.io DataGrid (type: "datagrid") where the grid key is the node name lowerCamelCase.
  - If multiple="false" and the node is a scalar (e.g., data-type="TEXT", "NUMBER", etc.), map to an appropriate input (textfield/number/email/checkbox/etc.) and set key = node name lowerCamelCase.
  - For nested structures, create nested components. If a parent has multiple children and should repeat, represent the parent as a DataGrid with inner components.
  - When mapping to input fields in the existing form, match by label semantics and intent. If an input already exists for a node, update its key to the datamodel node name lowerCamelCase and keep the field; otherwise insert a new field at a logical position.
  - Example: Node Address (multiple=false) with child Node Line (multiple=true, TEXT) => Create DataGrid key="address" with a textfield inside key="line".

Rules:
- Never remove the submit button or change unrelated fields.
- Use camelCase keys derived from datamodel node names.
- Keep labels concise.
- Ensure the final JSON is syntactically valid and minimal.
- It is NOT required to map every datamodel node. Prefer high-confidence matches to existing form fields. Only add new fields when the mapping is clear; leave ambiguous nodes unmapped (do not hallucinate fields).
"""


## User prompt (exact)

The following user message mirrors the request sent from the server route.


In [ ]:
import json

USER = (
    "Current Form.io JSON:\n" + json.dumps(CURRENT_JSON) +
    "\n\nDatamodel (XML):\n" + DATAMODEL_XML +
    "\n\nTask: Update the form JSON to reflect the datamodel, following the contract above and returning only the final JSON."
)



## Call sequence (abstract)

1. Receive HTTP POST with `currentJson` and `dataModelText`/`dataModelFileId`.
2. Load datamodel text when a fileId is provided.
3. Select a model (default: Anthropic Sonnet) and construct prompts.
4. Invoke the model with `SYSTEM` and `USER` messages.
5. Parse JSON from the response, accepting fenced JSON if needed.
6. Normalize to ensure `{ display: 'form', components: [...] }`.
7. Return `{ json, raw }` for debugging and downstream use.


## Illustrative provider call (pseudo-Python)

This example shows the request shape; replace with your preferred SDK.


In [ ]:
# Pseudo-code; not meant to run as-is
# from anthropic import Anthropic
# client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
# resp = client.messages.create(
#   model="claude-sonnet-4-0",
#   messages=[
#     {"role": "system", "content": SYSTEM},
#     {"role": "user", "content": USER},
#   ],
#   max_tokens=4000,
# )
# raw = resp.content[0].text

